# RGI15 (South Asia, East)

F. Maussion & S. Galos

In [ ]:
import pandas as pd
import geopandas as gpd
import subprocess
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import numpy as np
from utils import (mkdir, submission_summary, needs_size_filter, size_filter, plot_map, plot_date_hist, 
                   find_duplicates, open_zip_shapefile, correct_geoms, fix_overaps)
import os

### specify RGI-region and storage paths

In [ ]:
# Region of interest
reg = 15

# go down from rgi7_scripts/workflow
data_dir = '../../rgi7_data/'

# Level 2 GLIMS files
l2_dir = os.path.join(data_dir, 'l2_sel_reg_tars')

# Output directories
output_dir = mkdir(os.path.join(data_dir, 'l3_rgi7a'))
output_dir_tar = mkdir(os.path.join(data_dir, 'l3_rgi7a_tar'))

# RGI v6 file for comparison later 
rgi6_reg_file = os.path.join(data_dir, 'l0_RGIv6', '15_rgi60_SouthAsiaEast.zip')

# Region is based on GAMDAM, use for comparison
support_dir = os.path.join(data_dir, 'l0_support_data')
gamdam_dir = os.path.join(support_dir, 'gamdam')

### Load the input data

In [ ]:
# Read L2 files
shp = gpd.read_file('tar://' + l2_dir + f'/RGI{reg:02d}.tar.gz/RGI{reg:02d}/RGI{reg:02d}.shp')

### List of submissions 

In [ ]:
sdf, _ = submission_summary(shp)
sdf

In [ ]:
# # Optional: write out selection in intermediate shape files for manual GIS review
# tmp_output_dir = mkdir(os.path.join(data_dir, 'l0_tmp_data', f'rgi{reg:02d}_inventories'))
# tmp_output_dir_tar = mkdir(os.path.join(data_dir, 'l0_tmp_data'))
# for subid in shp.subm_id.unique():
#     s_loc = shp.loc[shp.subm_id == subid]
#     s_loc.to_file(tmp_output_dir + f'/subm_{int(subid):03d}.shp')
# print('Taring...')
# print(subprocess.run(['tar', '-zcvf', f'{tmp_output_dir_tar}/rgi{reg:02d}_inventories.tar.gz', '-C', 
#                       os.path.join(data_dir, 'l0_tmp_data'), f'rgi{reg:02d}_inventories']))

## Outline selection 

### Apply selection criteria to create the RGI-07 data subset

In [ ]:
# There are 10 ids in Sakai that are replaced by 9 Bolch outlines (one covers two)
replace_ids = [905769, 905912, 905983, 905984, 906004, 906092, 906107, 906241, 906286, 906319]

In [ ]:
# try to get the data relevant for RGI07 and select by attributes
rgi7 = shp.loc[shp['subm_id'].isin([752, 765])].copy()
print(len(rgi7))
rgi7 = rgi7.loc[~rgi7['anlys_id'].isin(replace_ids)]
print(len(rgi7))
rgi7['is_rgi6'] = False

In [ ]:
# Size filter?
needs_size_filter(rgi7)

### Some sanity checks 

In [ ]:
dupes = find_duplicates(rgi7)

In [ ]:
rgi7 = correct_geoms(rgi7)

In [ ]:
rgi7 = fix_overaps(rgi7)

In [ ]:
rgi7 = correct_geoms(rgi7)

In [ ]:
print(len(rgi7))
rgi7 = size_filter(rgi7)
print(len(rgi7))

In [ ]:
sdf, df_class = submission_summary(rgi7)
df_class

In [ ]:
# Check the orphaned rock outcrops
orphan_f = os.path.join(data_dir, 'l1_orphan_interiors', f'RGI{reg:02d}', f'RGI{reg:02d}.shp')
if os.path.exists(orphan_f):
    orphan_f = gpd.read_file(orphan_f)
    check = np.isin(rgi7.subm_id.unique(), orphan_f.subm_id.unique())
    if np.any(check):
        print(f'Orphan rock outcrops detected in subm_id {rgi7.subm_id.unique()[check]}')
        orphan_f['area'] = orphan_f.to_crs({'proj':'cea'}).area

### Plots 

In [ ]:
plot_map(rgi7, reg)

In [ ]:
plot_map(rgi7, reg, is_rgi6=True)

In [ ]:
plot_date_hist(rgi7, reg)

### Text for github

In [ ]:
fgh = sdf.T
fgh

In [ ]:
print(fgh.to_markdown(headers=np.append(['subm_id'], fgh.columns)))

## Write out and tar 

In [ ]:
dd = mkdir(f'{output_dir}/RGI{reg:02d}/', reset=True)

print('Writing...')
rgi7.to_file(dd + f'RGI{reg:02d}.shp')

print('Taring...')
print(subprocess.run(['tar', '-zcvf', f'{output_dir_tar}/RGI{reg:02d}.tar.gz', '-C', output_dir, f'RGI{reg:02d}']))

## New RGI-file created - Check result!

load reference data (here GAMDAM original) to enable comparison

In [ ]:
# Region file to select from gamdam
reg_file = os.path.join(data_dir, '00_rgi70_regions', '00_rgi70_O1Regions', '00_rgi70_O1Regions.shp')

In [ ]:
# specify path to reference data set
import glob
from utils import open_zip_shapefile
gam_files = glob.glob(gamdam_dir + '/*.zip')
df_ref = []
for gf in gam_files:
    df_ref.append(open_zip_shapefile(gf))

df_ref = pd.concat(df_ref).reset_index()

In [ ]:
# Calculate representative points for reference data
ref_rp = df_ref.representative_point()

# Make a dataframe out of it and add the original index to recover it later
ref_rp = ref_rp.to_frame('geometry')
ref_rp['orig_index'] = df_ref.index

# Read region file
reg_f = gpd.read_file(reg_file)

# Make the overlay with the RGI region of interest -> get the rep. points which are located inside the region boundaries
ref_intersect = gpd.overlay(ref_rp, reg_f.loc[reg_f.RGI_CODE == f'{reg:02d}'], how='intersection')

# Now select the entries which intersect from the original shape file (-> extract the polygons) 
df_ref = df_ref.loc[ref_intersect['orig_index'].values].copy()

### Number of elements (differences do not necessarily depict major problems)

In [ ]:
# Test only Sakai
rgi7_ = shp.loc[shp['subm_id'].isin([752])].copy()

In [ ]:
print('Number of glaciers in new RGI subset:', len(rgi7_))
print('Number of glaciers in reference data:', len(df_ref))
print('Difference:', len(rgi7_)-len(df_ref))

### Total area

In [ ]:
# add an area field to the selected GAMDAM table
df_ref['area'] = df_ref.to_crs({'proj':'cea'}).area

In [ ]:
# print and compare area values
Area_RGI = rgi7_['area'].sum() * 1e-6
print('Area RGI [km²]:', Area_RGI)
Area_ref = df_ref['area'].sum() * 1e-6
print('Area Ref [km²]:', Area_ref)
d = (Area_RGI - Area_ref)
print('Area difference [km²]:',d)